You've done all your processing and transformations. Now it's time to validate, package, and submit! (dont worry if you dont have some of the core measures, we will add a placeholder field with `Missing`). You also can check your errors and add to your external workflow by checking the `data/core-measures-test/report-summary.txt` file. 

Import the CoreMeasures object which provides the functionality for the 
core measure pipeline

In [13]:
from jdc_utils import CoreMeasures
from pathlib import Path
import os

In [14]:
datafolder = "../../data/external_workflow"

In [15]:
os.listdir(datafolder)

['baseline.csv', 'timepoints.csv']

In [16]:
Path(datafolder).resolve()/"baseline.csv"

WindowsPath('C:/Users/kranz-michael/projects/jdc-utilities/data/external_workflow/baseline.csv')

In [17]:
core_measures = CoreMeasures(transform_steps=["add_new_names","add_missing_fields"])
core_measures.add_baseline(Path(datafolder)/"baseline.csv")
core_measures.add_timepoints(Path(datafolder)/"timepoints.csv")

Now you have read in your core measure datasets (called `resources`):

In [18]:
core_measures.package.resource_names

['baseline', 'timepoints']

Validate and write your package to the standardized packaging format your core measures

In [19]:
core_measures.write("../../data/core-measures-test")

In [25]:
core_measures.write("../data/core-measures-test")

In [20]:
print(core_measures.written_package_report.to_summary())


# -----
# valid: data/baseline.csv 
# -----

## Summary 

+------------------------+-------------------+
| Description            | Size/Name/Count   |
+========================+===================+
| File name              | data/baseline.csv |
+------------------------+-------------------+
| File size (KB)         | 2018              |
+------------------------+-------------------+
| Total Time Taken (sec) | 0.043             |
+------------------------+-------------------+


# -----
# valid: data/timepoints.csv 
# -----

## Summary 

+------------------------+---------------------+
| Description            | Size/Name/Count     |
+========================+=====================+
| File name              | data/timepoints.csv |
+------------------------+---------------------+
| File size (KB)         | 27464               |
+------------------------+---------------------+
| Total Time Taken (sec) | 0.188               |
+------------------------+---------------------+




## Uploading and mapping data package to JDC

Once you've run created a validated data package,  you are ready to upload to the JDC. To upload (and update) a core measure you can then use the `.submit()` method. This will create a new version of the correspoinding file. Or, if it is your first time uploading and you do not have a file guid minted, run without a file guid to initiate new file record.

In [21]:
commons_project = "TEST"
file_guid = 'dg.6VTS/80b0372c-a90f-44f9-87a4-c699ed11d051' 
sheepdog_file_submitter_id = 'core-measures-test'
credentials_path="../../credentials.json"

In [22]:
core_measures.submit(
    commons_project_code=commons_project,
    commons_file_guid=file_guid, #comment out if creating a new version
    commons_file_submitter_id=sheepdog_file_submitter_id,
    commons_credentials_path=credentials_path
)

## Deleting test files and records

First grab the most recent records from indexd and sheepdog. Confirm this is indeed the file you just created and uploaded:

Note, need you need to update the file guid if wanting to test update function after creating a new record.

In [23]:
from gen3.auth import Gen3Auth
from gen3.submission import Gen3Submission
from gen3.index import Gen3Index
from gen3.file import Gen3File

index = Gen3Index(Gen3Auth(refresh_file=credentials_path))
sub = Gen3Submission(Gen3Auth(refresh_file=credentials_path))
files = Gen3File(Gen3Auth(refresh_file=credentials_path))
sheepdog_rec = sub.export_node("JCOIN","TEST","reference_file",fileformat="json")
index_rec = index.get_record(sheepdog_rec["data"][-1]["object_id"])

[2023-06-01 13:04:15,201][WARNING] failed to write token cache file: C:\Users\kranz-michael/.cache/gen3/token_cache_23fa6b0896aef79de0ce457aa59b87e0
[2023-06-01 13:04:15,202][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_23fa6b0896aef79de0ce457aa59b87e0.tmp_eraseme_61515_1685642655' -> 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_23fa6b0896aef79de0ce457aa59b87e0'
[2023-06-01 13:04:15,203][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x000001CB9FF90B20>, C:\Users\kranz-michael/.cache/gen3/token_cache_23fa6b0896aef79de0ce457aa59b87e0, eyJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV8yMDIwLTAxLTAyVDA2OjM1OjA4WiIsInR5cCI6IkpXVCJ9.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2pjb2luLmRhdGFjb21tb25zLmlvL3VzZXIiLCJhdWQiOlsiaHR0cHM6Ly9qY29pbi5kYXRhY29tbW9ucy5pby91c2VyIiwiZmVuY2UiLCJvcGVuaWQiLCJ1c2VyIiwiZGF0YSIsImFkbWluIiwiZ29vZ2xlX2xpbmsiLCJnb29nbGVfc2VydmljZV9hY2NvdW50IiwiZ29vZ2xlX2NyZW

Then delete all file locations and the sheepdog record:

In [24]:
files.delete_file_locations(index_rec["did"])
sub.delete_record("JCOIN","TEST",sheepdog_rec["data"][-1]["id"])

[2023-06-01 13:04:25,531][WARNING] failed to write token cache file: C:\Users\kranz-michael/.cache/gen3/token_cache_23fa6b0896aef79de0ce457aa59b87e0
[2023-06-01 13:04:25,532][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_23fa6b0896aef79de0ce457aa59b87e0.tmp_eraseme_46911_1685642665' -> 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_23fa6b0896aef79de0ce457aa59b87e0'
[2023-06-01 13:04:25,532][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x000001CB9FF916F0>, C:\Users\kranz-michael/.cache/gen3/token_cache_23fa6b0896aef79de0ce457aa59b87e0, eyJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV8yMDIwLTAxLTAyVDA2OjM1OjA4WiIsInR5cCI6IkpXVCJ9.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2pjb2luLmRhdGFjb21tb25zLmlvL3VzZXIiLCJhdWQiOlsiaHR0cHM6Ly9qY29pbi5kYXRhY29tbW9ucy5pby91c2VyIiwiZmVuY2UiLCJvcGVuaWQiLCJ1c2VyIiwiZGF0YSIsImFkbWluIiwiZ29vZ2xlX2xpbmsiLCJnb29nbGVfc2VydmljZV9hY2NvdW50IiwiZ29vZ2xlX2NyZW

<Response [200]>